### A little script to make a neat report from my Object Detection trials

In [1]:
import pickle #Needed to deserialize and load the object detection results from Disk
import pandas as pd #Needed to build the excel report

In [2]:
#We start with loading the data from disk and loading it to a Python object
pathd = "/Users/davidmoore/Downloads/e2eSensorNetwork-master/"
with open(pathd+"result.picke", "rb") as f:
    dump = pickle.load(f)
    
#dump represents a list of dictionaries
#[{'image': '/home/pi/Documents/yolo-object-detection/images/2020-12-22/11-17-55.jpg',
#  'result': ['car: 0.7074'],
#  'timing': '[INFO] YOLO took 10.105930 seconds'}]

#I would like this type of data in an Excel report please!

In [3]:
#we know that the dictionary has at least four pieces of data
#a) an image path
#b) a result with 'object' and 'confidence level'
#c) the time it took for the inference engine to deliver the detection
#I will split b) into Obj1 and Confobj1
#But also note that Yolo could detect multiple objects in a single photo

#lets start with what we know - these are the fields for the excel sheet right now
colNames=['Obj1','ConfObj1','InferTime','Image']

#but we need a function that tells us the maximum number of objects found in a given photo
#aka how many columns do we need in excel.  It is going to be at least four but could be many

def findSize(): #define a function to answer the how many question
    resultSize = 0 #start with zero objects
    for d in dump: #iterate over the list of dictionaries
        if len(d['result']) > resultSize: #if this dictionary has more objects than our running count
            resultSize = len(d['result']) #update our running count to that new maximum objects per photo
            
    return resultSize  #go ahead and give us the result as resultSize


#let us initialize some variables
resultSize = findSize() #we need to know the maximum number of objects per photo.  Min = 0
resArray = {} #resArray is the record which we set to blank
resList = [] #resList is the set of records we will create ( list )


#once we know the maximum number of columns needed - let's make decent english names
for n in range(2,resultSize+1):
    colNames.append('Obj'+str(n))
    colNames.append('ConfObj'+str(n))
  

#each object will be labelled Objx with the associated confidence labelled ConfObjx


#we need our columns in an order that makes sense to the Excel user
#so we need a sort key

def order(listing):
    if listing=="Image":
        return -2
    if listing=="InferTime":
        return -1
    else:
        return int(listing[len(listing)-1])

#order returns a signed integer from -2, -1, 1 ..  to resultSize
#we use this later to sort the column names for our Excel user


#next we need to iterate over the list of dictionaries and process them out
#so we will flatten the dictionaries into a 2d matrix and clean up
for d in dump:
    resArray = {} #empty record
    cnt=0
    for e in d['result']: #d['result'] contains an object label and confidence level
        cnt+=1
        items = e.split(':') #split by key value
        resArray[cnt] = items[0].strip().replace(' ','') #key is assigned 
        resArray[str(cnt)+'-conf'] = float(items[1]) #value is assigned
    timing = d['timing'].split(' ')[3] #let us retrieve the time from the string   
    resArray['timing'] = float(timing.strip()) #convert the time taken to a float   
    resArray['image'] = d['image']  #basic copy  
     
    resList.append(resArray) #add the record to the matrix   
      
        

In [4]:
df = pd.DataFrame(resList) #for convenience let us use Pandas with our 2D matrix

In [5]:
df.columns = colNames #over-ride the keys with more meaningful English Names   
    
colNames = sorted(colNames, reverse=False,key=order) #shuffle the columns in a more meaningful order  

df = df[colNames] #now shuffle the matrix in the desired order

In [6]:
df.info() #have a look at the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Image      192 non-null    object 
 1   InferTime  192 non-null    float64
 2   Obj1       90 non-null     object 
 3   ConfObj1   90 non-null     float64
 4   Obj2       6 non-null      object 
 5   ConfObj2   6 non-null      float64
 6   Obj3       1 non-null      object 
 7   ConfObj3   1 non-null      float64
dtypes: float64(4), object(4)
memory usage: 12.1+ KB


In [7]:
df.to_csv('data.csv') #finally drop the data out in CSV format.  Well I don't really LOVE Excel